# Web Scrapping on Flipcart.com for Mobile Data

### Step 1: Importing all necessary libraries.

In [7]:
import requests 
import bs4
from bs4 import BeautifulSoup
import re
import io

### Step 2: Send Request for data and getting the Response.

In [2]:
response = requests.get("https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off")
print(response)
requests.session().close() # if the output of this step is other than <Response [200]> so something went wrong.

<Response [200]>


### Step 3 : Getting the count for no. of times loop to be executed from no. of products we get in response.

In [5]:
soup = BeautifulSoup(response.content , "html.parser")
count = soup.find_all("span", attrs = {"class":"_2yAnYN"})

r = re.compile("[0-9]+,[0-9]+")
count = (re.findall(r , count[0].getText())[0]).split(",")
s = ""
count = int(s.join(count))
print(count)                  # Total no. of products.
count = count // 24           # Here 24 is the no. of products on per page.

print(count)                  # Finally the count for loop to be executed.
print(type(count))

11066
461
<class 'int'>


### Step 4: Create a .csv file to store data.

In [10]:
with io.open("MobileData.csv","w",encoding="utf8") as f1:
    f1.write("Title,Memory,Display,Camera,Battery,Processor,Selling_price,Cost_price,Rating,Discount\n")

### Step 5: Some functions we needed to clean the data.

In [13]:
def price(str):               # this function returns a price by removing ",'s" & " ₹ " this sign.
    s = ""
    l = str.split(",")
    for i in l:
        s += i
    return s[1:]
print(price('₹57,990'))

def processors(str):          # this function returns str by replacing ",'s" with " | ".
    s = ""
    if ',' in str:
        l = str.split(",")
        for i in l:
            s += i + "|"
        return s
    else:        
        return str
print(processors('Redmi 8 (Onyx Black, 64 GB)'))

57990
Redmi 8 (Onyx Black| 64 GB)|


### Step 6: Final stage of storeing data into the csv file.

In [11]:

for i in range (count + 1):
    i += 1
    url1 = "https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page="
    url1 += str(i)                    # Adding the page no. to the URL at the end.
    resp = requests.get(url1)         # requesting the data to URL and storing the response.
    requests.session().close()        # closeing the session.
    readfull = BeautifulSoup(resp.content , "html.parser")  #parseing the response data into html.

    cards = readfull.find_all("div",attrs={"class":"_1UoZlX"})  #list of products on the page.
    for card in cards:                # Looping through each product.
        disp = ""
        proc =""
        memo =""
        cam = ""
        battery =""
        discount =""
        title = card.find("div", attrs={"class":"_3wU53n"}).getText()
        Selling_price = card.find("div",attrs={"class":"_2rQ-NK"}).getText()
        Cost_price = card.find("div",attrs={"class":"_2GcJzG"})
        discount = card.find("div",attrs={"class":"VGWI6T"})
        if Cost_price == None:
            Cost_price = Selling_price
        else:
            Cost_price = Cost_price.getText()
        rating = card.find("div",attrs={"class":"hGSR34"})
        if rating == None:
            rating = ""
        else:
            rating = rating.getText()

        if discount == None:
            discount = ""
        else:
            discount = discount.getText()
            
        j = card.find("ul", attrs = {"class":"vFw0gD"})

        specs = j.find_all("li",attrs = {"class":"tVe95H"}) # list of Features of a product
        memo = specs[0].getText()
        for k in specs:                          # looping through each feature in features of a product.
            if k.getText().split(" ")[-1] == "Display":
                disp = k.getText()
            elif k.getText().split(" ")[-1] == "Processor" or "Processor" in k.getText().split(" ") :
                proc = k.getText()
            elif k.getText().split(" ")[-1] == "Camera":
                cam = k.getText()
            elif k.getText().split(" ")[-1] == "Battery":
                battery = k.getText()

        # Storeing the all the features into a variable data.
        data = processors(title) +","+ memo +","+ processors(disp) +","+ processors(cam) +","+ processors(battery) +","+ processors(proc) +","+ price(Selling_price) +","+ price(Cost_price) +","+ rating +","+ discount +"\n"
        
        with io.open("MobileData.csv","a",encoding="utf8") as f1:
            f1.write(data)  # appending the data into the csv file.

print("done")        

done


### Note : Please wait for a while after executing the Step 6 it takes some time to retrive and store data to the file, wait untill the done msg in output or an error.